### **1. 지하수 데이터 수집**

In [ ]:
#코드 8-11
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime, date, time

warnings.filterwarnings('ignore')

In [ ]:
#코드 8-12
path = './대전태평(암반)/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.xlsx')]
file_list_py

In [ ]:
#코드 8-13
df = pd.DataFrame()
for i in file_list_py:
  data = pd.read_excel(path + i)
  df = pd.concat([df, data])
df

In [ ]:
#코드 8-14
df = df.reset_index(drop=True)
df

In [ ]:
#코드 8-15
df.info()

In [ ]:
#코드 8-16
df['Date'] = pd.to_datetime(df['날짜'],format='%Y%m%d') +\
             pd.to_timedelta(df['시간'].astype(int),unit='h')

df.set_index(df['Date'], inplace=True)
df.head(3)

In [ ]:
#코드 8-17
df1 = df[['수온(℃)', '수위(el.m)','EC(㎲/㎝)']]
df1.columns = ['temp', 'level', 'EC']
df1.head(3)

In [ ]:
#코드 8-18
df1.isnull().sum()

In [ ]:
#코드 8-19
df1.to_csv('./대전지하수.csv', encoding='cp949')

### **2. 이상치 확인 및 제거**

In [ ]:
#코드 8-20
df = pd.read_csv('./대전지하수.csv', index_col='Date', parse_dates=True, encoding='cp949')
df.describe()

In [ ]:
#코드 8-21
df.hist(bins=50, figsize=(10,6))
plt.show()

In [ ]:
#코드 8-22
df.boxplot()

In [ ]:
#코드 8-23
plt.subplot(1, 3, 1)
df.boxplot(column='temp', return_type='both')
plt.subplot(1, 3, 2)
df.boxplot(column='level', return_type='both')
plt.subplot(1, 3, 3)
df.boxplot(column='EC', return_type='both')
plt.show()

In [ ]:
#코드 8-24
plt.subplot(3, 1, 1)
df['temp'].plot()
plt.subplot(3, 1, 2)
df['level'].plot()
plt.subplot(3, 1, 3)
df['EC'].plot()
plt.show()

In [ ]:
#코드 8-25
q3_level = df['level'].quantile(q=0.75)
q1_level = df['level'].quantile(q=0.25)
iqr_level = q3_level - q1_level
print(iqr_level)

In [ ]:
#코드 8-26
upper_level = q3_level + 1.5 * iqr_level
lower_level = q1_level - 1.5 * iqr_level
print(upper_level, '/', lower_level)
print((df['level'] > upper_level).sum())
print((df['level'] < lower_level).sum())

In [ ]:
#코드 8-27
df_iqr_level = df[(df['level'] < upper_level) & (df['level'] > lower_level)]
df_iqr_level['level'].plot()

In [ ]:
#코드 8-28
#사분위수와 IQR 구하기
q3_temp = df['temp'].quantile(q=0.75)
q1_temp = df['temp'].quantile(q=0.25)
iqr_temp = q3_temp-q1_temp
print('IQR:', iqr_temp)
#지하수온 상한값, 하한값, 이상치 개수 구하기
upper_temp = q3_temp + iqr_temp * 1.5
lower_temp = q1_temp - iqr_temp * 1.5
print(upper_temp, '/', lower_temp)
print((df['temp'] > upper_temp).sum())
print((df['temp'] < lower_temp).sum())

In [ ]:
#코드 8-29
#사분위수와 IQR 구하기
q3_ec = df['EC'].quantile(q=0.75)
q1_ec = df['EC'].quantile(q=0.25)
iqr_ec = q3_ec - q1_ec
print('IQR:', iqr_ec)
#지하수온 상한값, 하한값, 이상치 개수 구하기
upper_ec = q3_ec + iqr_ec * 1.5
lower_ec = q1_ec - iqr_ec * 1.5
print(upper_ec, '/', lower_ec)
print((df['EC'] > upper_ec).sum())
print((df['EC'] < lower_ec).sum())

In [ ]:
#코드 8-30
df_iqr = df[(df['temp'] < upper_temp) & (df['temp'] > lower_temp) &\
            (df['level'] < upper_level) & (df['level'] > lower_level) &\
             (df['EC'] < upper_ec) & (df['EC'] > lower_ec)]

In [ ]:
#코드 8-31
df_iqr.describe()

In [ ]:
#코드 8-32
df_iqr.hist(bins=50, figsize=(10,6))
plt.show()

In [ ]:
#코드 8-33
plt.subplot(1, 3, 1)
df_iqr.boxplot(column='temp', return_type='both')
plt.subplot(1, 3, 2)
df_iqr.boxplot(column='level', return_type='both')
plt.subplot(1, 3, 3)
df_iqr.boxplot(column='EC', return_type='both')
plt.show()

In [ ]:
#코드 8-34
plt.subplot(3, 1, 1)
df_iqr['temp'].plot()
plt.subplot(3, 1, 2)
df_iqr['level'].plot()
plt.subplot(3, 1, 3)
df_iqr['EC'].plot()
plt.show()

In [ ]:
#코드 8-35
df_iqr.to_csv('대전지하수_정제.csv', encoding='cp949')

### **3. 표준화와 정규화**

In [ ]:
#코드 8-36
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('./대전지하수_정제.csv')
df.head()

In [ ]:
#코드 8-37
df = df[['level', 'EC']]
df.describe()

In [ ]:
#코드 8-38
df.hist()
plt.show()

In [ ]:
#코드 8-39
df1 = df.copy()
df1.head(3)

In [ ]:
#코드 8-40
df1['level_z_score'] = (df1['level'] - df1['level'].mean()) / df1['level'].std()
df1['EC_z_score'] = (df1['EC'] - df1['EC'].mean()) / df1['EC'].std()
df1.head(3)

In [ ]:
#코드 8-41
df1.describe()

In [ ]:
#코드 8-42
df1.hist(figsize=(8,5))
plt.show()

In [ ]:
#코드 8-43
df2 = df.copy()
df2_standard = (df2-df2.mean()) / df2.std()
df2_standard.head(3)

In [ ]:
#코드 8-44
df3 = df.copy()

df3['level_minmax'] = (df3['level']-df3['level'].min()) / (df3['level'].max()\
                                                           -df3['level'].min())
df3['EC_minmax'] = (df3['EC'] - df3['EC'].min()) / (df3['EC'].max() - df3['EC'].min())
df3.head(3)

In [ ]:
#코드 8-45
df3.describe()

In [ ]:
#코드 8-46
df3.hist(figsize=(8,6))
plt.show()